In [ ]:
import pandas as pd

train_df = pd.read_csv("train_critic.csv").dropna()  # Carregando dados de treino
train_df = train_df.groupby('Sentiment', group_keys=False).apply(lambda x: x.sample(1500)).reset_index(drop=True)

In [ ]:
from transformers import AutoTokenizer, Trainer, Trainer, TrainingArguments, AutoModelForSequenceClassification
import torch

NUM_LABELS = 2  # Número de classes
model_name = "google-bert/bert-base-uncased"  # Escolha o modelo conforme sua necessidade
tokenizer = AutoTokenizer.from_pretrained(model_name)  # Carregar o tokenizer
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=NUM_LABELS)  # Carregar o modelo de classificação

In [ ]:
from datasets import Dataset
import tensorflow as tf
import random
import numpy as np
tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)


train_data_set = Dataset.from_pandas(train_df)
train_val_set = train_data_set.train_test_split (test_size = 0.2)

print(train_val_set)

x_train = train_val_set["train"]["Review"]
x_val = train_val_set["test"]["Review"]
y_train = train_val_set["train"]["Sentiment"]
y_val = train_val_set["test"]["Sentiment"] 
